#  Introduction to Operations Research -  Linear Programming

In this notebook I will demonstrate how to use the [Julia Programming Language](https://julialang.org) and the powerful [JuMP](https://jump.dev) optimization framework to solve common business problems using Operations Research techniques.  The examples in this notebook are borrowed from __Introduction to Operations Research -  Fredrick S. Hillier, Gerald J Lieberman 10th edition__

This notebook assumes you are somewhat familiar with basic Julia syntax and things like functions, macros, packages, etc.

If you don't have the packages installed in the next cell, uncomment and run.

In [3]:
# Using Pkg
# Pkg.add("Clp")
# Pkg.add("Cbc")
# Pkg.add("GLPK")

In [4]:
# Lets begin by importing the necessary packages
using JuMP
using GLPK #GLPK is an open source optimizing software

## Problem #1 Setup

>The WYNDOR GLASS CO. produces high quality glass products, including windows and glass doors. It has three plants. Aluminum frames and hardware are made in Plant 1, wood frames are made in Plant 2, and Plant 3 produces the glass and assembles the products.
>
>Because of declining earnings, top management has decided to revamp the company's product line. Unprofitable products are being discontinued, releasing production capacity to launch two new products having large sales potential:
>* Product 1: An 8-foot glass door with aluminum framing
>* Product 2: A 4x6 foot double-hung wood-framed window
>
>Product 1 requires some of the production capacity in Plants 1 and 3, but none in Plant 2. Product 2 needs only Plants 2 and 3. The marketing division has concluded that the company could sell as much of either product as could be produced by these plants. However, because both products would be competing for the same production capacity in Plant 3, its not clear which mix of the two products would be *most profitable*.
>
>**Determine what the production rates should be for the two products in order to maximize their total profit, subject to the restrictions imbosed by the limited production capacities available in the three plants. (each product will be produced in batches of 20, so the production rate is defined as the number of batches produced per week.) Any combination of production rates that satisfies these restrictions is permitted, including producing none of one product and as much as possible of the other.**


> The Operations Research team has identified the data that need to be gathered:
1. Number of hours of production time available per week in each plant for the products.
1. Number of hours of production time used in each plant for each batch produced of the new product
1. Profit per batch produced of each new product.

### Data

The Operations Research Team discovers that Plant 1 has only 4 hours of available production time per week, Plant 2 has 12, and Plant 3 has 18. Additionally, they learn that a batch of 20 units (the minimum batch size) takes 1 hour at Plant 1 and 3 hours at Plant 3. Product 2 takes 2 hours at Plant 2 and 2 hours at Plant 3.  Finally they learn that the Profit per batch of product is 3,000 for Product 1 and 5,000 for product 2. The data is summarized in the table below.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky">Plant</th>
    <th class="tg-0pky">Product 1 Production Time</th>
    <th class="tg-0pky">Product 2 Production time</th>
    <th class="tg-0pky">Availabel time per week</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">1</td>
    <td class="tg-0pky">1</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">4</td>
  </tr>
  <tr>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">12</td>
  </tr>
  <tr>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">18</td>
  </tr>
  <tr>
    <td class="tg-0pky">Profit</td>
    <td class="tg-0pky">3000</td>
    <td class="tg-0pky">5000</td>
    <td class="tg-0pky"></td>
  </tr>
</tbody>
</table>

In [ ]:
m = Model(with_optimizer(GLPK.Optimizer))

In [42]:
# Define the variables used in our model.
# x₁ is the number of batches of product 1
# x₂ is the number of batches of product 2
# We only want to deal with full batches
# so we set the type to Int
@variable(m, x₁ ≥ 0, Int)
@variable(m, x₂ ≥ 0, Int)

# Now we define our objective. We wish to maximize
# our profit Z = 3x₁ + 5x₂ in thousands of dollars
@objective(m, Max, 3x₁ + 5x₂)

# Finally, lets specify our constraints. Production capacity.
@constraint(m, x₁ ≤ 4)
@constraint(m, 2x₂ ≤ 12)
@constraint(m, 3x₁ + 2x₂ ≤ 18)

3 x₁ + 2 x₂ ≤ 18.0

In [43]:
# Now just call optimize!() which modifies our model, m, in place
optimize!(m)

In [53]:
println(m)
println("The Optimal Values of xᵢ: ")
println("x₁ = ", value(x₁))
println("x₂ = ", value(x₂))

Max 3 x₁ + 5 x₂
Subject to
 x₁ ≤ 4.0
 2 x₂ ≤ 12.0
 3 x₁ + 2 x₂ ≤ 18.0
 x₁ ≥ 0.0
 x₂ ≥ 0.0
 x₁ integer
 x₂ integer

The Optimal Values of xᵢ: 
x₁ = 2.0
x₂ = 6.0


What we have just solved is called a Linear Integer Program. *Linear* because our objective and constraints are linear functions of our variables $xᵢ$ (conventionally referred to as *decision variables*. *Integer* because we only want solutions that are integer values of our variables.  Typically integer programs are significantly more difficult to solve than standard linear programs, but with the size of this model and Julia's performance, it's nothing to sweat about.

Integer programs are common in manufacturing where we gain no value in producing half a car, or launching 1/3 of a satellite into space.

Also notice that we have a nonnegativity constraint on our decision variables as it is not meaningful to discuss making negative quantities of product.

## Problem #2 setup

>Operations Research optimization techniques are also commonly used in medicine.  Take for example radiation therapy delivered to prostate cancer patients.  The treatment is typically delivered by directing beams of radiation at the cancerous tissue.  Because the tumor is surrounded by non cancerous tissue, there is a risk of damaging the healthy tissue with the radiation beam.  In this example we will lay out a simple version of the more complex model used in real life.
>
>>E. K. Lee and M. Zaider, “Operations Research Advances Cancer Therapeutics,” Interfaces, 38(1): 5–25, Jan.–Feb. 2008.
>
>Imagine that our patient Thomas has been diagnosed with prostate cancer. Given the location of the tumor on the prostate you use your best judgement to place two available radiation therapy beams so as to minimize the beam travel through healthy tissue.  Both beams are important because you want to attack the tumor from multiple angles to maximize treatment delivery.  The problem, however, is that each beam delivers different doses of radiation to the tumor and to the surrounding healthy tissue.
>
>Your objective is to choose the optimal combination of beam dosages so as to achieve the target dose in the tumor, while minimizing doses to surrounding tissue.
>
>The treatment region is broken down into 4 main areas:
>1. Healthy anatomy
>1. Critical tissues
>1. Tumor region
>1. Center of tumor

### Data
The relevant absorbtion dosages and constraints are summarized in the table below

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg" style="undefined;table-layout: fixed; width: 500px">
<colgroup>
<col style="width: 50px">
<col style="width: 50px">
<col style="width: 50px">
<col style="width: 50px">
</colgroup>
<thead>
  <tr>
    <th class="tg-0pky"></th>
    <th class="tg-0pky" colspan="2">Fraction of Entry Dose Absorbed by Area</th>
    <th class="tg-0pky"></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-baqh">Area</td>
    <td class="tg-0lax">Beam 1</td>
    <td class="tg-0lax">Beam 2</td>
    <td class="tg-0lax">Restriction on Total Dose, Kilorads</td>
  </tr>
  <tr>
    <td class="tg-0pky">Healthy anatomy</td>
    <td class="tg-0pky">0.4</td>
    <td class="tg-0pky">0.5</td>
    <td class="tg-0pky">Minimize</td>
  </tr>
  <tr>
    <td class="tg-0pky">Critical tissues</td>
    <td class="tg-0pky">0.3</td>
    <td class="tg-0pky">0.1</td>
    <td class="tg-0pky">&le; 2.7</td>
  </tr>
  <tr>
    <td class="tg-0pky">Tumor region</td>
    <td class="tg-0pky">0.5</td>
    <td class="tg-0pky">0.5</td>
    <td class="tg-0pky">= 6</td>
  </tr>
  <tr>
    <td class="tg-0pky">Center of tumor</td>
    <td class="tg-0pky">0.6</td>
    <td class="tg-0pky">0.4</td>
    <td class="tg-0pky">&ge; 6</td>
  </tr>
</tbody>
</table>

In [62]:
# Lets begin converting this problem into a JuMP model

m2 = Model(with_optimizer(GLPK.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [63]:
# Again, we'll begin by declaring our variables, our objective, and finally our constraints.
# For this problem, we'll allow fractional dosages.

@variable(m2, x₁ ≥ 0)
@variable(m2, x₂ ≥ 0)

@objective(m2, Min, 0.4x₁ + 0.5x₂)

@constraint(m2, 0.3x₁ + 0.1x₂ ≤ 2.7) #This constrain says we wont accept more than 2.7 Kilorads to critical tissues
@constraint(m2, 0.5x₁ + 0.5x₂ == 6) # this constraint specifies we want an exact total dose delivered to the tumor region
@constraint(m2, 0.6x₁ + 0.4x₂ ≥ 6) # This says we want the center of the tumor to get more than the general tumor region

0.6 x₁ + 0.4 x₂ ≥ 6.0

In [65]:
optimize!(m2)
println(m2)
println("The Optimal Values of xᵢ: ")
println("x₁ = ", value(x₁))
println("x₂ = ", value(x₂))

Min 0.4 x₁ + 0.5 x₂
Subject to
 0.5 x₁ + 0.5 x₂ = 6.0
 0.6 x₁ + 0.4 x₂ ≥ 6.0
 0.3 x₁ + 0.1 x₂ ≤ 2.7
 x₁ ≥ 0.0
 x₂ ≥ 0.0

The Optimal Values of xᵢ: 
x₁ = 7.500000000000002
x₂ = 4.499999999999997


Here we see we've identified the optimal values up to numerical precision is (7.5, 4.5).  Unlike Example 1, which is a resource allocation problem, this problem is a cost/benefit optimization problem. Specifically, we are trying to minimze the cost of achieving a specific dosage delivered to the tumor.